In [1]:
import numpy as np
import pandas as pd

In [2]:
# READ DATA
movies_raw_df = pd.read_csv(r'C:\Users\Julia Chang\Downloads\archive.zip')
movies_raw_df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [3]:
# FILTER DATA
movies_to_select = ((movies_raw_df['Genre'] == 'horror') &
                   # Restrict to American movies.
                   (movies_raw_df['Origin/Ethnicity'] == 'American') &
                   # Only movies from 2000.
                   (movies_raw_df['Release Year'] > 1999))

In [4]:
horror_df = movies_raw_df[movies_to_select]['Plot']
horror_df.head()

13617    In November 1999, tourists and fans of The Bla...
13640    Matthew Van Helsing, the alleged descendant of...
13681    A small group of fervent Roman Catholics belie...
13731    Cotton Weary, now living in Los Angeles and th...
13763    Amy Mayfield, a student at a prestigious film ...
Name: Plot, dtype: object

In [5]:
horror_df.shape

(260,)

In [6]:
# PREPROCESS DATA
# Join all plots into a string.
horror_str = horror_df.str.cat(sep=' ')

In [7]:
import spacy

# Load language model.
nlp = spacy.load('en_core_web_sm', disable = ['parser', 'tagger', 'ner'])

# Extract tokens (words).
def get_tokens(doc_text):
    # This pattern is a modification of the defaul filter from the
    # Tokenizer() object in keras.preprocessing.text. 
    # It just indicates which patters no skip.
    skip_pattern = '\r\n \n\n \n\n\n!"-#$%&()--.*+,-./:;<=>?@[\\]^_`{|}~\t\n\r '
    
    tokens = [token.text.lower() for token in nlp(doc_text) if token.text not in skip_pattern]
    
    return tokens

# Get tokens.
tokens = get_tokens(horror_str)
# Let us see the first tokens.
tokens[0:9]

C:\Users\Julia Chang\anaconda3\Lib\site-packages\spacy\pipeline\lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


['in', 'november', '1999', 'tourists', 'and', 'fans', 'of', 'the', 'blair']

In [8]:
len(tokens)

165891

In [9]:
# FEATURE EXTRACTION
len_0 = 25
tokens[0:len_0] # features
tokens[len_0:len_0+1] # target

['resident']

In [10]:
# Generate sequences.
train_len = len_0 + 1
text_sequences = []
for i in range(train_len, len(tokens)):
    # Construct sequence.
    seq = tokens[i - train_len: i]
    # Append.
    text_sequences.append(seq)

In [11]:
# First five sequences
for i in range(0,5):
    print(' '.join(text_sequences[i]))
    print('-----')

in november 1999 tourists and fans of the blair witch project descend on the small town of burkittsville maryland where the film is set local resident
-----
november 1999 tourists and fans of the blair witch project descend on the small town of burkittsville maryland where the film is set local resident jeff
-----
1999 tourists and fans of the blair witch project descend on the small town of burkittsville maryland where the film is set local resident jeff a
-----
tourists and fans of the blair witch project descend on the small town of burkittsville maryland where the film is set local resident jeff a former
-----
and fans of the blair witch project descend on the small town of burkittsville maryland where the film is set local resident jeff a former psychiatric
-----


In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [13]:
# VECTORIZATION
# Encode character sequences as numerical features
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences) # creates dictionary with numbers
# Get numeric sequences
sequences = tokenizer.texts_to_sequences(text_sequences) # each sentence uses a dictionary so there's a list of numbers
sequences[0]

[8,
 12576,
 12575,
 2397,
 2,
 12574,
 5,
 1,
 5560,
 630,
 2195,
 2932,
 20,
 1,
 452,
 157,
 5,
 12573,
 7486,
 42,
 1,
 117,
 7,
 362,
 231,
 2933]

In [14]:
vocabulary_size = len(tokenizer.word_counts) # vocabulary size = # unique tokens
sequences = np.array(sequences) # store sequences in a numpy array

In [15]:
# X-Y SPLIT
from tensorflow.keras.utils import to_categorical
X = sequences[:,:-1] # take all the rows, take all the columns except the last one
X

array([[    8, 12576, 12575, ...,     7,   362,   231],
       [12576, 12575,  2397, ...,   362,   231,  2933],
       [12575,  2397,     2, ...,   231,  2933,   297],
       ...,
       [   20,     4,  1551, ...,    22,     1,    59],
       [    4,  1551,  1684, ...,     1,    59,     5],
       [ 1551,  1684,    22, ...,    59,     5,     6]])

In [16]:
X.shape

(165865, 25)

In [17]:
seq_len = X.shape[1]
y = sequences[:,-1] # select all last row indices
y

array([2933,  297,    4, ...,    5,    6,  169])

In [18]:
# Convert to categorical (we add +1 because Keras needs a placeholder.)
y = to_categorical(y, num_classes=(vocabulary_size + 1))
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [26]:
# MODEL DEFINITION
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding

def create_model(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(input_dim=vocabulary_size,
                       output_dim=seq_len))
    model.add(LSTM(units=50, return_sequences=True))
    model.add(LSTM(units=50))
    model.add(Dense(units=50, activation='relu'))
    model.add(Dense(units=vocabulary_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    # Build the model by providing an input shape
    model.build(input_shape=(None, seq_len)) # model needed to be explicitly built before calling model.summary()
    
    model.summary()
    
    return model

In [27]:
import tensorflow as tf
print(tf.__version__)

2.17.0


In [30]:
# Create model and see summary
model = create_model(vocabulary_size+1, seq_len)

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ (None, 25, 25)              │         314,425 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ (None, 25, 50)              │          15,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_9 (LSTM)                        │ (None, 50)                  │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 50)                  │           2,550 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 12577)               │         641,427 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 993,802 (3.79 MB)

 Trainable params: 993,802 (3.79 MB)

 Non-trainable params: 0 (0.00 B)

In [31]:
# MODEL FIT
model.fit(x=X, y=y, batch_size=128, epochs=10, verbose=1)
# get model metrics.
loss, accuracy = model.evaluate(x=X, y=y)
print(f'Loss: {loss}\nAccuracy: {accuracy}')

Epoch 1/10
1296/1296 ━━━━━━━━━━━━━━━━━━━━ 56s 39ms/step - accuracy: 0.0647 - loss: 7.2953
Epoch 2/10
1296/1296 ━━━━━━━━━━━━━━━━━━━━ 48s 37ms/step - accuracy: 0.0771 - loss: 6.5872
Epoch 3/10
1296/1296 ━━━━━━━━━━━━━━━━━━━━ 46s 35ms/step - accuracy: 0.0929 - loss: 6.3643
Epoch 4/10
1296/1296 ━━━━━━━━━━━━━━━━━━━━ 52s 40ms/step - accuracy: 0.1156 - loss: 6.1360
Epoch 5/10
1296/1296 ━━━━━━━━━━━━━━━━━━━━ 45s 35ms/step - accuracy: 0.1233 - loss: 5.9537
Epoch 6/10
1296/1296 ━━━━━━━━━━━━━━━━━━━━ 48s 37ms/step - accuracy: 0.1276 - loss: 5.8355
Epoch 7/10
1296/1296 ━━━━━━━━━━━━━━━━━━━━ 51s 39ms/step - accuracy: 0.1348 - loss: 5.7235
Epoch 8/10
1296/1296 ━━━━━━━━━━━━━━━━━━━━ 47s 36ms/step - accuracy: 0.1385 - loss: 5.6290
Epoch 9/10
1296/1296 ━━━━━━━━━━━━━━━━━━━━ 69s 53ms/step - accuracy: 0.1420 - loss: 5.5509
Epoch 10/10
1296/1296 ━━━━━━━━━━━━━━━━━━━━ 55s 43ms/step - accuracy: 0.1471 - loss: 5.4444
5184/5184 ━━━━━━━━━━━━━━━━━━━━ 46s 9ms/step - accuracy: 0.1497 - loss: 5.3639
Loss: 5.3398036956787

In [32]:
# Model Fit 2
model.fit(x=X, y=y, batch_size=200, epochs=10, verbose=1)
loss, accuracy = model.evaluate(x=X, y=y)
print(f'Loss: {loss}\nAccuracy: {accuracy}')

Epoch 1/10
830/830 ━━━━━━━━━━━━━━━━━━━━ 42s 50ms/step - accuracy: 0.1506 - loss: 5.3500
Epoch 2/10
830/830 ━━━━━━━━━━━━━━━━━━━━ 35s 42ms/step - accuracy: 0.1519 - loss: 5.3052
Epoch 3/10
830/830 ━━━━━━━━━━━━━━━━━━━━ 39s 46ms/step - accuracy: 0.1534 - loss: 5.2540
Epoch 4/10
830/830 ━━━━━━━━━━━━━━━━━━━━ 40s 48ms/step - accuracy: 0.1572 - loss: 5.1929
Epoch 5/10
830/830 ━━━━━━━━━━━━━━━━━━━━ 38s 45ms/step - accuracy: 0.1581 - loss: 5.1482
Epoch 6/10
830/830 ━━━━━━━━━━━━━━━━━━━━ 44s 53ms/step - accuracy: 0.1595 - loss: 5.0935
Epoch 7/10
830/830 ━━━━━━━━━━━━━━━━━━━━ 43s 52ms/step - accuracy: 0.1638 - loss: 5.0409
Epoch 8/10
830/830 ━━━━━━━━━━━━━━━━━━━━ 39s 48ms/step - accuracy: 0.1627 - loss: 4.9971
Epoch 9/10
830/830 ━━━━━━━━━━━━━━━━━━━━ 38s 45ms/step - accuracy: 0.1673 - loss: 4.9386
Epoch 10/10
830/830 ━━━━━━━━━━━━━━━━━━━━ 38s 46ms/step - accuracy: 0.1695 - loss: 4.9024
5184/5184 ━━━━━━━━━━━━━━━━━━━━ 48s 9ms/step - accuracy: 0.1730 - loss: 4.8321
Loss: 4.817480564117432
Accuracy: 0.17212

In [33]:
# Model Fit 3
model.fit(x=X, y=y, batch_size=500, epochs=10, verbose=1)
loss, accuracy = model.evaluate(x=X, y=y)
print(f'Loss: {loss}\nAccuracy: {accuracy}')

Epoch 1/10
332/332 ━━━━━━━━━━━━━━━━━━━━ 40s 117ms/step - accuracy: 0.1733 - loss: 4.8144
Epoch 2/10
332/332 ━━━━━━━━━━━━━━━━━━━━ 38s 113ms/step - accuracy: 0.1755 - loss: 4.7853
Epoch 3/10
332/332 ━━━━━━━━━━━━━━━━━━━━ 42s 128ms/step - accuracy: 0.1782 - loss: 4.7605
Epoch 4/10
332/332 ━━━━━━━━━━━━━━━━━━━━ 44s 133ms/step - accuracy: 0.1770 - loss: 4.7527
Epoch 5/10
332/332 ━━━━━━━━━━━━━━━━━━━━ 43s 129ms/step - accuracy: 0.1773 - loss: 4.7329
Epoch 6/10
332/332 ━━━━━━━━━━━━━━━━━━━━ 37s 112ms/step - accuracy: 0.1791 - loss: 4.7219
Epoch 7/10
332/332 ━━━━━━━━━━━━━━━━━━━━ 48s 143ms/step - accuracy: 0.1803 - loss: 4.6896
Epoch 8/10
332/332 ━━━━━━━━━━━━━━━━━━━━ 46s 138ms/step - accuracy: 0.1827 - loss: 4.6734
Epoch 9/10
332/332 ━━━━━━━━━━━━━━━━━━━━ 42s 125ms/step - accuracy: 0.1852 - loss: 4.6477
Epoch 10/10
332/332 ━━━━━━━━━━━━━━━━━━━━ 40s 119ms/step - accuracy: 0.1861 - loss: 4.6320
5184/5184 ━━━━━━━━━━━━━━━━━━━━ 43s 8ms/step - accuracy: 0.1928 - loss: 4.5736
Loss: 4.565622806549072
Accurac

In [36]:
# Model Fit 4
model.fit(x=X, y=y, batch_size=500, epochs=100, verbose=1)
loss, accuracy = model.evaluate(x=X, y=y)
print(f'Loss: {loss}\nAccuracy: {accuracy}')

Epoch 1/100
332/332 ━━━━━━━━━━━━━━━━━━━━ 25s 73ms/step - accuracy: 0.2008 - loss: 4.4791
Epoch 2/100
332/332 ━━━━━━━━━━━━━━━━━━━━ 24s 73ms/step - accuracy: 0.1982 - loss: 4.4756
Epoch 3/100
332/332 ━━━━━━━━━━━━━━━━━━━━ 24s 73ms/step - accuracy: 0.2034 - loss: 4.4381
Epoch 4/100
332/332 ━━━━━━━━━━━━━━━━━━━━ 25s 74ms/step - accuracy: 0.2067 - loss: 4.4027
Epoch 5/100
332/332 ━━━━━━━━━━━━━━━━━━━━ 25s 74ms/step - accuracy: 0.2067 - loss: 4.3867
Epoch 6/100
332/332 ━━━━━━━━━━━━━━━━━━━━ 24s 74ms/step - accuracy: 0.2115 - loss: 4.3637
Epoch 7/100
332/332 ━━━━━━━━━━━━━━━━━━━━ 25s 74ms/step - accuracy: 0.2124 - loss: 4.3486
Epoch 8/100
332/332 ━━━━━━━━━━━━━━━━━━━━ 24s 73ms/step - accuracy: 0.2133 - loss: 4.3335
Epoch 9/100
332/332 ━━━━━━━━━━━━━━━━━━━━ 24s 73ms/step - accuracy: 0.2164 - loss: 4.3053
Epoch 10/100
332/332 ━━━━━━━━━━━━━━━━━━━━ 24s 74ms/step - accuracy: 0.2157 - loss: 4.3038
Epoch 11/100
332/332 ━━━━━━━━━━━━━━━━━━━━ 24s 74ms/step - accuracy: 0.2187 - loss: 4.2768
Epoch 12/100
332/33

In [37]:
# Model Fit 5
model.fit(x=X, y=y, batch_size=500, epochs=700, verbose=1)
loss, accuracy = model.evaluate(x=X, y=y)
print(f'Loss: {loss}\nAccuracy: {accuracy}')

Epoch 1/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 25s 73ms/step - accuracy: 0.3287 - loss: 3.3711
Epoch 2/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 25s 75ms/step - accuracy: 0.3288 - loss: 3.3751
Epoch 3/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 24s 73ms/step - accuracy: 0.3292 - loss: 3.3682
Epoch 4/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 24s 73ms/step - accuracy: 0.3320 - loss: 3.3529
Epoch 5/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 24s 73ms/step - accuracy: 0.3333 - loss: 3.3457
Epoch 6/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 24s 74ms/step - accuracy: 0.3335 - loss: 3.3401
Epoch 7/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 25s 74ms/step - accuracy: 0.3366 - loss: 3.3255
Epoch 8/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 25s 74ms/step - accuracy: 0.3354 - loss: 3.3271
Epoch 9/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 25s 75ms/step - accuracy: 0.3353 - loss: 3.3267
Epoch 10/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 33s 99ms/step - accuracy: 0.3367 - loss: 3.3184
Epoch 11/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 25s 75ms/step - accuracy: 0.3391 - loss: 3.3044
Epoch 12/700
332/33

332/332 ━━━━━━━━━━━━━━━━━━━━ 31s 93ms/step - accuracy: 0.4219 - loss: 2.7622
Epoch 133/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 30s 90ms/step - accuracy: 0.4207 - loss: 2.7688
Epoch 134/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 35s 106ms/step - accuracy: 0.4242 - loss: 2.7589
Epoch 135/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 31s 93ms/step - accuracy: 0.4197 - loss: 2.7727
Epoch 136/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 37s 111ms/step - accuracy: 0.4245 - loss: 2.7482
Epoch 137/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 28s 84ms/step - accuracy: 0.4235 - loss: 2.7535
Epoch 138/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 28s 85ms/step - accuracy: 0.4255 - loss: 2.7400
Epoch 139/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 28s 85ms/step - accuracy: 0.4256 - loss: 2.7398
Epoch 140/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 34s 103ms/step - accuracy: 0.4248 - loss: 2.7406
Epoch 141/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 31s 92ms/step - accuracy: 0.4276 - loss: 2.7248
Epoch 142/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 35s 104ms/step - accuracy: 0.4233 - loss: 2.7440
Epoch 143

332/332 ━━━━━━━━━━━━━━━━━━━━ 71s 212ms/step - accuracy: 0.4686 - loss: 2.4716
Epoch 263/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 63s 189ms/step - accuracy: 0.4710 - loss: 2.4626
Epoch 264/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 64s 191ms/step - accuracy: 0.4755 - loss: 2.4346
Epoch 265/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 60s 180ms/step - accuracy: 0.4753 - loss: 2.4406
Epoch 266/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 50s 150ms/step - accuracy: 0.4766 - loss: 2.4369
Epoch 267/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 54s 162ms/step - accuracy: 0.4716 - loss: 2.4475
Epoch 268/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 60s 179ms/step - accuracy: 0.4749 - loss: 2.4349
Epoch 269/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 59s 175ms/step - accuracy: 0.4746 - loss: 2.4375
Epoch 270/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 68s 205ms/step - accuracy: 0.4758 - loss: 2.4334
Epoch 271/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 54s 163ms/step - accuracy: 0.4771 - loss: 2.4280
Epoch 272/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 52s 156ms/step - accuracy: 0.4747 - loss: 2.4431
Ep

332/332 ━━━━━━━━━━━━━━━━━━━━ 39s 118ms/step - accuracy: 0.5028 - loss: 2.2772
Epoch 393/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 40s 120ms/step - accuracy: 0.5010 - loss: 2.2885
Epoch 394/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 39s 118ms/step - accuracy: 0.5006 - loss: 2.2892
Epoch 395/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 39s 118ms/step - accuracy: 0.5074 - loss: 2.2646
Epoch 396/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 39s 118ms/step - accuracy: 0.5078 - loss: 2.2655
Epoch 397/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 40s 120ms/step - accuracy: 0.5080 - loss: 2.2601
Epoch 398/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 39s 119ms/step - accuracy: 0.5074 - loss: 2.2600
Epoch 399/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 39s 118ms/step - accuracy: 0.5081 - loss: 2.2547
Epoch 400/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 40s 119ms/step - accuracy: 0.5107 - loss: 2.2490
Epoch 401/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 40s 119ms/step - accuracy: 0.5061 - loss: 2.2656
Epoch 402/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 40s 122ms/step - accuracy: 0.5053 - loss: 2.2685
Ep

332/332 ━━━━━━━━━━━━━━━━━━━━ 35s 105ms/step - accuracy: 0.5272 - loss: 2.1470
Epoch 523/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 35s 106ms/step - accuracy: 0.5263 - loss: 2.1588
Epoch 524/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 38s 114ms/step - accuracy: 0.5255 - loss: 2.1567
Epoch 525/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 38s 115ms/step - accuracy: 0.5253 - loss: 2.1545
Epoch 526/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 36s 109ms/step - accuracy: 0.5286 - loss: 2.1455
Epoch 527/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 35s 104ms/step - accuracy: 0.5291 - loss: 2.1426
Epoch 528/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 36s 107ms/step - accuracy: 0.5258 - loss: 2.1559
Epoch 529/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 37s 111ms/step - accuracy: 0.5294 - loss: 2.1389
Epoch 530/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 35s 106ms/step - accuracy: 0.5294 - loss: 2.1433
Epoch 531/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 37s 113ms/step - accuracy: 0.5304 - loss: 2.1305
Epoch 532/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 38s 116ms/step - accuracy: 0.5307 - loss: 2.1372
Ep

332/332 ━━━━━━━━━━━━━━━━━━━━ 51s 152ms/step - accuracy: 0.5467 - loss: 2.0489
Epoch 653/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 45s 136ms/step - accuracy: 0.5482 - loss: 2.0450
Epoch 654/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 41s 123ms/step - accuracy: 0.5497 - loss: 2.0354
Epoch 655/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 43s 128ms/step - accuracy: 0.5451 - loss: 2.0535
Epoch 656/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 48s 143ms/step - accuracy: 0.5363 - loss: 2.0976
Epoch 657/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 43s 129ms/step - accuracy: 0.5398 - loss: 2.0728
Epoch 658/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 44s 133ms/step - accuracy: 0.5418 - loss: 2.0702
Epoch 659/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 41s 123ms/step - accuracy: 0.5437 - loss: 2.0602
Epoch 660/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 42s 126ms/step - accuracy: 0.5503 - loss: 2.0419
Epoch 661/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 43s 129ms/step - accuracy: 0.5453 - loss: 2.0446
Epoch 662/700
332/332 ━━━━━━━━━━━━━━━━━━━━ 41s 122ms/step - accuracy: 0.5482 - loss: 2.0380
Ep

In [38]:
# SAVE MODEL
from pickle import dump
dump(tokenizer, open('tokenizer','wb'))
model.save('model.h5')

In [40]:
from tensorflow.keras.models import load_model
model = load_model('model.h5') # reloading the model

In [53]:
# GENERATE NEW TEXT
from keras.preprocessing.sequence import pad_sequences
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    # Set seed_text as input_text.
    input_text = seed_text
    output_text = input_text.split() # initialize output_text with the seed_text split into words
    
    for i in range(num_gen_words):
        # Encode input text.
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        pred_probs = model.predict(pad_encoded, verbose=0)
        pred_word_ind = np.argmax(pred_probs, axis=-1)[0]
        pred_word = tokenizer.index_word[pred_word_ind]
        input_text += ' ' + pred_word
        output_text.append(pred_word)
    return ' '.join(output_text)

In [50]:
# EXAMPLE 1
sample_text = horror_df.iloc[100][:383]
print(sample_text)

Officer Frank Williams (Steven Vidler) and his partner Blaine investigate an abandoned house, where they find a young woman with her eyes ripped out. A large figure with an axe then murders Blaine and Frank has his arm chopped off before he is able to shoot the attacker in the head. Afterwards, detectives find seven bodies in the house, all of which have had their eyes ripped out.


In [51]:
seed_text = sample_text[:190]
print(seed_text)

Officer Frank Williams (Steven Vidler) and his partner Blaine investigate an abandoned house, where they find a young woman with her eyes ripped out. A large figure with an axe then murders 


In [55]:
generated_text = generate_text(model, tokenizer, seq_len, seed_text, 40)
print(seed_text + ' ' + generated_text + '...')

Officer Frank Williams (Steven Vidler) and his partner Blaine investigate an abandoned house, where they find a young woman with her eyes ripped out. A large figure with an axe then murders  Officer Frank Williams (Steven Vidler) and his partner Blaine investigate an abandoned house, where they find a young woman with her eyes ripped out. A large figure with an axe then murders blaine and frank is expelled and to move back to the hospital but is unable to handle the cellphone chips and notices a campsite and runs to talk to the bathroom and destroys his restraints as being disarmed doing with...


In [56]:
# EXAMPLE 2
seed_text2 = 'the film starts in a dark house where a group of teenagers friends meet to spend the weekend when they suddenly hear'

In [57]:
generated_text2 = generate_text(model, tokenizer, seq_len, seed_text, 80)
print(seed_text2 + ' ' + generated_text2 + '...')

the film starts in a dark house where a group of teenagers friends meet to spend the weekend when they suddenly hear Officer Frank Williams (Steven Vidler) and his partner Blaine investigate an abandoned house, where they find a young woman with her eyes ripped out. A large figure with an axe then murders blaine and frank is expelled and to move back to the hospital but is unable to handle the cellphone chips and notices a campsite and runs to talk to the bathroom and destroys his restraints as being disarmed doing with a way to accept that bession was driven drugs by the cabin but survives the truth about foot as a miscarriage and the group are cut the gun and the two militia members are seen to the ground in the...


In [59]:
# EXAMPLE 3
seed_text3 = movies_raw_df[movies_raw_df['Genre'] == 'comedy']['Plot'].iloc[330]
print(seed_text3)

Cocky college football star Francis Finnegan has his eye on the attractive Gloria van Dayham, as does his rival, Larry Stacey.
Francis gets a job in a department store owned by Stacey's father, where salesgirl June Cort develops an attraction to him. Finnegan proposes that Stacey's store sponsor a football team, which causes rival shop owner Whimple to do likewise. The team's head cheerleader, Mimi, falls for team mascot Joe, meanwhile, and everybody pairs off with the perfect partner after the big game.


In [60]:
generated_text3 = generate_text(model, tokenizer, seq_len, seed_text3, 90)
print(seed_text3 + ' ' + generated_text3 + '...')

Cocky college football star Francis Finnegan has his eye on the attractive Gloria van Dayham, as does his rival, Larry Stacey.
Francis gets a job in a department store owned by Stacey's father, where salesgirl June Cort develops an attraction to him. Finnegan proposes that Stacey's store sponsor a football team, which causes rival shop owner Whimple to do likewise. The team's head cheerleader, Mimi, falls for team mascot Joe, meanwhile, and everybody pairs off with the perfect partner after the big game. Cocky college football star Francis Finnegan has his eye on the attractive Gloria van Dayham, as does his rival, Larry Stacey. Francis gets a job in a department store owned by Stacey's father, where salesgirl June Cort develops an attraction to him. Finnegan proposes that Stacey's store sponsor a football team, which causes rival shop owner Whimple to do likewise. The team's head cheerleader, Mimi, falls for team mascot Joe, meanwhile, and everybody pairs off with the perfect partner 

In [62]:
# ADDING TEMPERATURE PARAMETER

def generate_text_temp(model, tokenizer, seq_len, seed_text, num_gen_words, temperature):
    output_text = []
    input_text = seed_text
    for i in range(num_gen_words):
        # Encode input text.
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        # Add if the input text does not have length len_0.
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        # Get learned distribution.
        pred_distribution = model.predict(pad_encoded, verbose=0)[0]
        
        # Apply temperature transformation.
        new_pred_distribution = np.power(pred_distribution, (1/temperature))
        new_pred_distribution = new_pred_distribution / new_pred_distribution.sum()
        
        # Sample from modified distribution.
        choices = range(new_pred_distribution.size)
        
        pred_word_ind = np.random.choice(a=choices, p=new_pred_distribution)
        
        # Convert from numeric to word.
        pred_word = tokenizer.index_word[pred_word_ind]
        # Attach predicted word.
        input_text += ' ' + pred_word
        # Append new word to the list.
        output_text.append(pred_word)
        
    return ' '.join(output_text)        

In [64]:
generated_text4 = generate_text_temp(model, tokenizer, seq_len, seed_text2, 82, 0.9)
print(seed_text2 + ' ' + generated_text4 + '...')

the film starts in a dark house where a group of teenagers friends meet to spend the weekend when they suddenly hear frankie who once she had been discharged revealing the officers pick winston who raised when mathias uses david 's fingers into a blood of killing their bodies while grady rolls henry she ordered to suffer from didi destroying bed of his wrist supposed on newly confused and 1953 ziegler out of a miscarriage and attacked he her own fault ashley crenna and candles is responsible and lived attacking the mistress 's key into the living family but she is murdered he and...


In [67]:
generated_text5 = generate_text_temp(model, tokenizer, seq_len, seed_text2, 82, 0.5)
print(seed_text2 + ' ' + generated_text5 + '...')

the film starts in a dark house where a group of teenagers friends meet to spend the weekend when they suddenly hear frankie who are made by the monster are attracting bends in washington sets progresses the bye rage but jumps heavily ends and the curse is unable to deny 100 jake appears to battle as their torsos as she slowly stops princess reaper and destroy the firefly residence as a protective measure from the infirmary wydell spent exasperated commits but gone and interrogated with the stairs to cooperate to get out to rescue lisa 's crafted bullets eleanor pauline wakes through hades africa...


In [68]:
generated_text6 = generate_text_temp(model, tokenizer, seq_len, seed_text2, 82, 0.1)
print(seed_text2 + ' ' + generated_text6 + '...')

the film starts in a dark house where a group of teenagers friends meet to spend the weekend when they suddenly hear frankie who are made by the fbi seven test is heard in a wheelchair while finishing markus clippings deer of the nanny and is maimed by smoking out of the advice kreeg finds the burnt of then buds filled with used at 1408 to 18 unhinged jason in a patrol officer in the hospital where she is strapped to the car stock manley her wound in the insistence mental monster reports the paint alive wants to visit carl scales the group saying...
